In [44]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

In [90]:
"""
tid = tournament id 
pid = player id
[beers = number of beers drunken] => unnecessary, due to players are only on the list if they at least drink one bear at a tournament

players = defaultdict(float)

players["10"] = .32
players["15"] = 6.32
players["14"] = 9.32
players["13"] = 6.322
players["11"] = 79.32

print(players)

https://thispointer.com/python-pandas-how-to-create-dataframe-from-dictionary/ => use players.items()
pd.DataFrame(players.items(),columns=["pid","ema"]).sort_values("ema",ascending=False)[0:2]
"""

df = pd.read_csv("tally.csv",usecols=['pid','tid'])
df = df.sort_values('tid')
df[:10]

,pid,tid
1314,23,172
1321,57,172
1320,59,172
1319,46,172
1318,43,172
1317,35,172
1316,33,172
1315,25,172
1313,22,172
1312,21,172


In [102]:
class Classifier:
    def __init__(self,N,alpha,df):
        self.N = N
        self.alpha = alpha
        self.df = df
    
    def fit(self,tids):
        #mapping from playerid => ema value
        players = defaultdict(float)
        
        # iterate over every tournament
        for i in np.unique(tids):
            
            # that tournament
            tournament_data = self.df[self.df.tid == i]
            
            # dont know how good the quality of the data is, but this should not happen anyways
            # if len(tournament_data):
                
            # add value to every player (regardless of his beer count)
            for pid in tournament_data.pid:
                
                # update ema
                players[int(pid)] = self.alpha * 1 + (1-self.alpha) * players[int(pid)]
        
        self.fitted = pd.DataFrame(players.items(),columns=["pid","ema"])
        
    def predict(self):
        # return last N player with highest ema score
        return self.fitted.sort_values("ema",ascending=False)[:self.N]
        
    def score(self,x):
        return len(set(x)-set(self.predict()))

In [103]:
clf = Classifier(30,0.16,df)

scores = []

for tid in sorted(set(df.tid)):
    clf.fit(df[df.tid<tid])
    scores.append(clf.score(df[df.tid == tid+1].pid))

pd.Series(scores).mean()    

12.96195652173913